In [1]:
#google drive mount to access data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
#library imports
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp
tfd = tfp.distributions
tfpl = tfp.layers
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
#image size and paths
IMAGE_SIZE = [224, 224]

ct_train_path = '/content/drive/MyDrive/lung_cancer_ct_scans_dataset/train'
ct_test_path = '/content/drive/MyDrive/lung_cancer_ct_scans_dataset/test'
path_train_path = '/content/drive/MyDrive/lung_cancer_pathology_dataset_updated/train'
path_test_path = '/content/drive/MyDrive/lung_cancer_pathology_dataset_updated/test'

In [5]:
#define transformations for images
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [6]:
#training and test sets for ct and pathology images
ct_training_set = train_datagen.flow_from_directory(ct_train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

ct_test_set = test_datagen.flow_from_directory(ct_test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

path_training_set = train_datagen.flow_from_directory(path_train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

path_test_set = test_datagen.flow_from_directory(path_test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1221 images belonging to 3 classes.
Found 174 images belonging to 3 classes.
Found 1221 images belonging to 3 classes.
Found 174 images belonging to 3 classes.


In [7]:
#multimodal data generator
def multimodal_generator(ct_generator, path_generator):
    while True:
        ct_images, ct_labels = ct_generator.next()
        path_images, path_labels = path_generator.next()
        yield [ct_images, path_images], ct_labels

In [8]:
#bayesian convolutional layers
class BayesianConv2D(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, activation=tf.nn.relu, **kwargs):
        super(BayesianConv2D, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.activation = activation

    def build(self, input_shape):
        self.kernel_posterior = self.add_weight(
            name='kernel_posterior',
            shape=(self.kernel_size[0], self.kernel_size[1], input_shape[-1], self.filters),
            initializer='random_normal',
            trainable=True
        )
        self.bias_posterior = self.add_weight(
            name='bias_posterior',
            shape=(self.filters,),
            initializer='random_normal',
            trainable=True
        )

    def call(self, inputs):
        output = tf.nn.conv2d(inputs, self.kernel_posterior, strides=[1, 1, 1, 1], padding='SAME')
        output = tf.nn.bias_add(output, self.bias_posterior)
        return self.activation(output)

In [9]:
#bayesian dense layers
class BayesianDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=tf.nn.relu, **kwargs):
        super(BayesianDense, self).__init__(**kwargs)
        self.units = units
        self.activation = activation

    def build(self, input_shape):
        self.kernel_posterior = self.add_weight(
            name='kernel_posterior',
            shape=(input_shape[-1], self.units),
            initializer='random_normal',
            trainable=True
        )
        self.bias_posterior = self.add_weight(
            name='bias_posterior',
            shape=(self.units,),
            initializer='random_normal',
            trainable=True
        )

    def call(self, inputs):
        output = tf.matmul(inputs, self.kernel_posterior)
        output = tf.nn.bias_add(output, self.bias_posterior)
        return self.activation(output)

In [17]:
#multimodal architecture
def create_multimodal_bcnn(input_shape):
    #ct scan path
    input1 = Input(shape=input_shape)
    x1 = BayesianConv2D(32, (3, 3))(input1)
    x1 = MaxPooling2D((2, 2))(x1)
    x1 = BayesianConv2D(64, (3, 3))(x1)
    x1 = MaxPooling2D((2, 2))(x1)

    #pathology images path
    input2 = Input(shape=input_shape)
    x2 = BayesianConv2D(32, (3, 3))(input2)
    x2 = MaxPooling2D((2, 2))(x2)
    x2 = BayesianConv2D(64, (3, 3))(x2)
    x2 = MaxPooling2D((2, 2))(x2)

    merged = Concatenate()([x1, x2])
    flattened = Flatten()(merged)

    x = BayesianDense(128)(flattened)
    x = BayesianDense(64)(x)

    output = Dense(3, activation='softmax')(x)

    model = Model(inputs=[input1, input2], outputs=output)
    return model

In [18]:
#create model
model = create_multimodal_bcnn(input_shape=(224, 224, 3))
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
#model summary
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 bayesian_conv2d_8 (Bayesia  (None, 224, 224, 32)         896       ['input_5[0][0]']             
 nConv2D)                                                                                         
                                                                                                  
 bayesian_conv2d_10 (Bayesi  (None, 224, 224, 32)         896       ['input_6[0][0]']       

In [20]:
#train model
history = model.fit(
    multimodal_generator(ct_training_set, path_training_set),
    steps_per_epoch=len(ct_training_set),
    epochs=50,
    validation_data=multimodal_generator(ct_test_set, path_test_set),
    validation_steps=len(ct_test_set),
)

KeyboardInterrupt: 

In [ ]:
#plot accuracy and loss over epochs
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['loss'], label = 'loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')